In [ ]:

import os
import time
import numpy as np
import cv2 as cv
import mediapipe as mp
from matplotlib import pyplot as plt

# Constants
DELAY = 50
INPUT_TYPES = ['palm', 'fist', 'thumbsup', 'gun', 'call']
MIN_INPUT_COUNT = 2000 + DELAY
INPUT_PATH = 'Inputs/'

# Make directories
def make_dirs():
    if not os.path.isdir(INPUT_PATH):
        os.mkdir(INPUT_PATH, mode=511)
    os.chdir(INPUT_PATH)
    for dirs in INPUT_TYPES:
        if not os.path.isdir(dirs):
            os.mkdir(dirs, mode=511)
    os.chdir('..')
